In [61]:
import os
import logging
import requests
import json
from tqdm.notebook import tqdm
from time import sleep
import functools
import msgpack

BEST_PLAYERS_URL = 'https://ch.tetr.io/api/users/lists/league?limit=100'
BEST_PLAYERS_FILE = 'best-players.json'

RECENT_GAMES_URL = 'https://ch.tetr.io/api/streams/league_userrecent_{}'  # fill in player id (found in best player list)
RECENT_GAMES_FILE = 'recent-games/recent-games-{}.json'  # fill in player id


# GAME_DOWNLOAD_URL = 'https://tetr.io/api/games/{}'  # fill in replay id (found in game history)

REPLAY_URL = 'https://inoue.szy.lol/api/replay/{}'  # fill in the replay id
REPLAY_FILE = 'replays/replay-{}-{}.ttr'  # fill in the user id and replay id

HEADERS = {'User-Agent': 'requests/2.28.1 <project-name>', 'From': '<name>/<email>'}

logging.getLogger().setLevel(logging.DEBUG)

In [62]:
def _load_from_cache(useCachedIfAvailable, file_path):
    if useCachedIfAvailable:
        if os.path.isfile(file_path):
            with open(file_path) as f:
                return json.load(f)


def _download_file(url, cache_file, **kwargs):
    r = requests.get(
        url,
        allow_redirects=True, 
        headers=HEADERS,
        **kwargs
    )

    with open(cache_file, 'wb+') as f:
        f.write(r.content)

    return json.loads(r.content)

In [63]:
def download_best_players(useCachedIfAvailable=True):
    data = _load_from_cache(useCachedIfAvailable, BEST_PLAYERS_FILE)
    
    if data is not None:
        data = data['data']['users']
        
        logging.info('[download_best_players] fetched best player list from local cache...')
    else:
        data = _download_file(BEST_PLAYERS_URL, BEST_PLAYERS_FILE)['data']['users']
        
        logging.info('[download_best_players] downloaded best player list from tetra channel...')
        
    return data

    
# download_best_players()[0]['username']
# download_best_players()

In [66]:
def download_recent_games(player_id, useCachedIfAvailable=True):
    cache_file = RECENT_GAMES_FILE.format(player_id)
    download_uri = RECENT_GAMES_URL.format(player_id)
    
    data = _load_from_cache(useCachedIfAvailable, cache_file)
    
    if data is not None:
        data = data['data']['records']
        
        logging.info(f'[download_recent_games] fetched recent games list of player {player_id} from local cache...')
    else:
        data = _download_file(download_uri, cache_file)['data']['records']
        
        logging.info(f'[download_recent_games] downloaded recent games list of player {player_id} from tetra channel...')
        
    return data            
    
# download_best_players()[0]['username']
# download_recent_games('5e7cbb652932b46c9c671ce4')

In [78]:
def download_replay(user_id, replay_id, useCachedIfAvailable=True):
    cache_file = REPLAY_FILE.format(user_id, replay_id)
    download_uri = REPLAY_URL.format(replay_id)
    
    data = _load_from_cache(useCachedIfAvailable, cache_file)
    
    if data is not None:
        logging.info(f'[download_replay] fetched replay {replay_id} from local cache...')
    else:
        logging.info(f'[download_replay] downloading replay {replay_id} from inoue bot...')
        
        r = requests.get(
            download_uri,
            allow_redirects=True,
            stream=True,
            headers=HEADERS
        )

        with open(cache_file, 'wb+') as f:
            total_length = int(r.headers.get('content-length'))
            
            for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1, leave=False):
                if chunk:
                    f.write(chunk)
                    f.flush()
        
            f.seek(0)
            data = json.load(f)
    
    return data
            
    
# lelz = download_replay('5e7cbb652932b46c9c671ce4', '638c5a12a28f8610572907cf')

INFO:root:[download_replay] fetched replay 638c5a12a28f8610572907cf from local cache...


lel


In [88]:
class GameIterator:
    def __init__(self):
        self.best_players = download_best_players()[0:3]
        self.games = {}
        self.current_player = 0
        self.current_game = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        max_games = len(self._get_games())
        
        if self.current_game >= max_games:
            self.current_player += 1
            self.current_game = 0
            
            if self.current_player >= len(self.best_players):
                raise StopIteration
        
        current_game = self._get_game()
        
        self.current_game += 1
        
        return self.best_players[self.current_player], current_game
    
    
    def _get_games(self):
        current_player_id = self.best_players[self.current_player]['_id']
        
        # fetch the game, if not fetched already
        if current_player_id not in self.games:
            self.games[current_player_id] = download_recent_games(current_player_id)
            
        return self.games[current_player_id]
    
    def _get_game(self):
        return self._get_games()[self.current_game]
        
    
    def has_next(self):
        temp_curr_player = self.current_player
        max_games = len(self._get_games())
        
        if self.current_game >= max_games:
            temp_curr_player += 1
            
        if temp_curr_player >= len(self.best_players):
            return False
        
        return True


In [ ]:
g = GameIterator()

with tqdm(total=10*100) as bar:
    while g.has_next():
        player, game = next(g)
        print(player['username'], game['replayid'])

        download_replay(player['_id'], game['replayid'])
        
        bar.update(1)
        sleep(60)
    

INFO:root:[download_best_players] fetched best player list from local cache...


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:root:[download_recent_games] fetched recent games list of player 5e7cbb652932b46c9c671ce4 from local cache...


icly 638c5758a28f8610572906c6
icly 638c53daa28f8610572905a7
icly 638c50eca28f861057290498
icly 638c3fe8df20761071f99799
icly 638c3cf8df20761071f99695
icly 638c39c877ac79105d2fe2c7
icly 638c360977ac79105d2fe12b
icly 638c332477ac79105d2fdffa


INFO:root:[download_recent_games] fetched recent games list of player 615fb20fe17beeef45104302 from local cache...


blaarg 638c5a12a28f8610572907cf
blaarg 638c5758a28f8610572906c6
blaarg 638c53daa28f8610572905a7
blaarg 638c50eca28f861057290498
blaarg 638c4eb3bf44ce1066aeca43
blaarg 638c4bd277ac79105d2fe94e
blaarg 638c49d977fc831073531a16
blaarg 638c460277fc8310735318a3
blaarg 638c428d1a8e44107adf7984
blaarg 638c3fe8df20761071f99799


INFO:root:[download_recent_games] fetched recent games list of player 5e844b0868270e617d52c990 from local cache...


czsmall0402 638cca08a28f8610572930dd
czsmall0402 638390fd9c7242cc85584c15
czsmall0402 637a58316412ae312aea05ce
czsmall0402 63712dd2efc473f7c4faec0e
czsmall0402 6367de71f9ffbe7fac581676
czsmall0402 635eac80456cd10bae7d4980
czsmall0402 635576d963d2855fefa11ae1
czsmall0402 634c21361aa72f8664d9d0f6
czsmall0402 63430225840b8fe69dce54d3
czsmall0402 6339b009d5a4ef5ffbfa3db0
